In [ ]:
import kagglehub
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from gtts import gTTS
from pathlib import Path
from IPython.display import Audio

# Path to the dataset (update this if needed)
dataset_path = Path("/root/.cache/kagglehub/datasets/shanks0465/braille-character-dataset/versions/1")

# Load the dataset: Traverse the directory and find all image files
image_files = []
for subdir, _, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_files.append(os.path.join(subdir, file))

# Verify if images were found
if image_files:
    print(f"Loaded {len(image_files)} images")
else:
    print("No images found. Please verify the dataset structure and file types.")

# Function to load and preprocess the dataset
def load_data(dataset_path):
    X = []
    y = []
    allowed_extensions = ['.png', '.jpg', '.jpeg']

    # Traverse the dataset folder
    for subdir, _, files in os.walk(dataset_path):
        for img_file in files:
            img_path = os.path.join(subdir, img_file)
            # Check if it's an image
            if any(img_file.lower().endswith(ext) for ext in allowed_extensions):
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Warning: Could not load image {img_path}")
                    continue

                # Labeling: Assuming the subdirectory name is the label (you can adjust this)
                label = subdir.split('/')[-1].lower()  # Use the folder name as the label

                # Preprocess image: Resize and flatten
                img = cv2.resize(img, (28, 28))  # Resize image to 28x28
                img = img.flatten()  # Flatten the image to a 1D array
                X.append(img)
                y.append(label)
            else:
                print(f"Skipping non-image file: {img_path}")

    # Ensure data is loaded
    if len(X) == 0:
        print("No images loaded. Please check the dataset path and file structure.")
    else:
        print(f"Loaded {len(X)} images")

    return np.array(X), np.array(y)

# Load the dataset
X, y = load_data(dataset_path)

# Proceed if data was successfully loaded
if X.shape[0] > 0:
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize classifiers
    knn = KNeighborsClassifier(n_neighbors=3)
    decision_tree = DecisionTreeClassifier(random_state=42)

    # Combine the classifiers into an ensemble using Voting Classifier
    voting_clf = VotingClassifier(estimators=[('knn', knn), ('dt', decision_tree)], voting='hard')
    voting_clf.fit(X_train, y_train)

    # Make predictions using the ensemble model
    y_pred = voting_clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Ensemble Model Accuracy: {accuracy * 100:.2f}%")

    # Convert the prediction to speech using gTTS
    def text_to_speech(text, output_file="output.mp3"):
        tts = gTTS(text=text, lang='en', slow=False)
        tts.save(output_file)
        print(f"Speech saved to {output_file}")

    # Test the model with a sample image
    test_image = X_test[0]
    predicted_text = voting_clf.predict([test_image])[0]
    print(f"Predicted Braille symbol: {predicted_text}")

    # Convert the predicted text to speech and save the audio
    text_to_speech(predicted_text, output_file="output.mp3")

    # Display the audio output
    display(Audio('output.mp3'))
else:
    print("No data available for training. Exiting...")


No images found. Please verify the dataset structure and file types.
No images loaded. Please check the dataset path and file structure.
No data available for training. Exiting...


In [ ]:
!pip install gtts